In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from collections import namedtuple

class BrowserAutomation:
    def __init__(self):
        self.browser_driver = webdriver.Chrome()
        self.browser_driver.maximize_window()
        self.browser_driver.implicitly_wait(10)
        self.browser_driver.get("https://app-staging.nokodr.com/")

    def terminate_browser(self):
        if self.browser_driver:
            self.browser_driver.quit()

    def locate_element_with_wait(self, selector, timeout=10, clickable=False):
        wait_instance = WebDriverWait(self.browser_driver, timeout)
        condition = EC.element_to_be_clickable if clickable else EC.presence_of_element_located
        return wait_instance.until(condition(selector))

    def navigate_to_signup(self):
        register_button = self.locate_element_with_wait((By.LINK_TEXT, "Sign up"), clickable=True)
        register_button.click()

    def fill_signup_form(self, user_email, agree_to_terms=True):
        email_field = self.locate_element_with_wait(
            (By.XPATH, "(//input[@class='slds-input ng-untouched ng-pristine ng-valid'])[3]")
        )
        email_field.clear()
        email_field.send_keys(user_email)

        if agree_to_terms:
            terms_checkbox = self.locate_element_with_wait((By.CLASS_NAME, "slds-checkbox__label"), clickable=True)
            terms_checkbox.click()

    def submit_form(self):
        register_button = self.locate_element_with_wait(
            (By.XPATH, "//div[@class='slds-col slds-size_1-of-1']"), clickable=True
        )
        register_button.click()

    def validate_success_message(self, expected_message):
        try:
            success_msg = self.locate_element_with_wait((By.XPATH, "//*[contains(text(),'OTP')]"))
            actual_text = success_msg.text
            if expected_message in actual_text:
                print(f"Verification Passed: Found '{expected_message}' in success message")
            else:
                print(f"Verification Failed: Expected '{expected_message}', but got '{actual_text}'")
        except TimeoutException:
            print("Verification Failed: Success message did not appear within the time limit")

    def validate_error_message(self, expected_error):
        try:
            # Check for unexpected success message
            WebDriverWait(self.browser_driver, 2).until(
                EC.presence_of_element_located((By.XPATH, "//*[contains(text(),'OTP')]"))
            )
            print(f"Error: Success message appeared when expecting error '{expected_error}'")
        except TimeoutException:
            # Look for error messages
            error_msgs = self.browser_driver.find_elements(By.CLASS_NAME, "error-message")
            error_texts = [error.text for error in error_msgs if error.text]
            if any(expected_error in message for message in error_texts):
                print(f"Error Validation Passed: Found '{expected_error}' in error messages")
            else:
                print(f"Error Validation Failed: Expected '{expected_error}', but found {error_texts or 'no errors'}")

class SignupTestSuite:
    def run_test_case(self, user_email, accept_terms, test_case_type, expected_result):
        automation = BrowserAutomation()
        try:
            automation.navigate_to_signup()
            automation.fill_signup_form(user_email, accept_terms)
            automation.submit_form()

            if test_case_type == "success":
                automation.validate_success_message(expected_result)
            elif test_case_type == "error":
                automation.validate_error_message(expected_result)
        except Exception as e:
            print(f"Test encountered an issue: {e}")
        finally:
            automation.terminate_browser()

            
# Define test case structure using namedtuple
TestCase = namedtuple('TestCase', ['case_name', 'user_email', 'accept_terms', 'case_type', 'expected_result'])

# Define test scenarios as a list of named tuples
test_cases = [
    TestCase(
        case_name="Successful Registration",
        user_email="valid.email@example.com",
        accept_terms=True,
        case_type="success",
        expected_result="OTP"
    ),
    TestCase(
        case_name="Invalid Email Format",
        user_email="invalid-email",
        accept_terms=True,
        case_type="error",
        expected_result="Invalid email format"
    ),
    TestCase(
        case_name="Missing Email",
        user_email="",
        accept_terms=True,
        case_type="error",
        expected_result="Email is required"
    ),
    TestCase(
        case_name="Terms Not Accepted",
        user_email="valid.email@example.com",
        accept_terms=False,
        case_type="error",
        expected_result="You must accept the terms and conditions"
    )
]

# Execute tests
if __name__ == "__main__":
    test_runner = SignupTestSuite()
    for case in test_cases:
        print(f"\nRunning test: {case.case_name}...")
        test_runner.run_test_case(
            user_email=case.user_email,
            accept_terms=case.accept_terms,
            test_case_type=case.case_type,
            expected_result=case.expected_result
        )
